In [1]:
!pip install folium
!pip install wget

  Preparing metadata (setup.py) ... - done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=7e3e6600d3e9e52594cb59055db7294e5a09860db6dbef0f160ea59c518a85d2
  Stored in directory: /root/.cache/pip/wheels/e1/e8/db/ebe4dcd7d7d11208c1e4e4ef246cea4fcc8d463c93405a6555
Successfully built wget


In [2]:
import pandas as pd
import wget
import folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [4]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
launch_sites_df.loc[len(launch_sites_df)] = ['NASA JSC', 29.559684888503615, -95.0830971930759]

In [6]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745
4,NASA JSC,29.559685,-95.083097


#### Mark all launch sites on a map

In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4.5)

for index in list(launch_sites_df.index):
    # Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
    coordinate = [launch_sites_df['Lat'][index],launch_sites_df['Long'][index]]
    site = launch_sites_df['Launch Site'][index]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True)\
        .add_child(folium.Popup(site))
    
    # Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
    marker = folium.map.Marker(
        coordinate,
        # Create an icon as a text label
        icon = DivIcon(
            icon_size=(20,20), icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [8]:
spacex_df['marker_color'] = 'green'
spacex_df.loc[spacex_df['class'] == 0,'marker_color'] = 'red'

#### Mark the success/failed launches for each site on the map

In [9]:
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame, create a Marker object with its coordinate
# customize the Marker's icon property to indicate if this launch was successed or failed
for i, row in spacex_df.iterrows():
    coordinate = [row['Lat'],row['Long']]
    marker = folium.map.Marker(
        coordinate,
        # Create an icon as a text label
        icon = folium.Icon(color='white', icon_color=row['marker_color']        )
    )
    marker_cluster.add_child(marker)

site_map

#### Calculate the distances between a launch site to its proximities

In [10]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map